In [7]:
!pip3 install ultralytics easyocr rapidfuzz supervision levenshtein
!pip3 uninstall opencv-python -y
!pip3 uninstall opencv-contrib-python -y
!pip3 install opencv-contrib-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 40.2 MB/s eta 0:00:00:00:0100:01
Found existing installation: opencv-python 4.10.0.84
Uninstalling opencv-python-4.10.0.84:
  Successfully uninstalled opencv-python-4.10.0.84
Found existing installation: opencv-contrib-python 4.10.0.84
Uninstalling opencv-contrib-python-4.10.0.84:
  Successfully uninstalled opencv-contrib-python-4.10.0.84
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (68.7 MB)


In [2]:
!pip3 install paddlepaddle-gpu "paddleocr>=2.0.1"

  Using cached paddlepaddle_gpu-2.6.1-cp311-cp311-manylinux1_x86_64.whl.metadata (8.6 kB)
  Using cached paddleocr-2.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached lmdb-1.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached fire-0.6.0.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.7 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 49.8 MB/

In [11]:
!pip3 install pandas
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import easyocr
from rapidfuzz.distance import JaroWinkler
import supervision as sv
from tqdm import tqdm
from Levenshtein import distance as levenshtein_distance
from paddleocr import PaddleOCR
import pandas as pd

In [13]:
upscale_model = cv2.dnn_superres.DnnSuperResImpl_create()
upscale_model.readModel('/notebooks/models/ESPCN_x4.pb')
upscale_model.setModel('espcn', 4)

In [32]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True, gpu_mem=1000, show_log=False)

In [4]:
import smtplib
from email.mime.text import MIMEText
sender_email = "shravanapps44@gmail.com"
receiver_email = "6092127855@vtext.com"  # For Verizon SMS-to-email
password = "pxyzvfpyghrduhmu"
def send_msg(process, num):

    # Your email credentials
    

    message = MIMEText(f"{num}% Done")
    message['Subject'] = f"{process}"
    message['From'] = sender_email
    message['To'] = receiver_email

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())

    print("Email sent successfully")

In [30]:
import numpy as np
from scipy import fft
from skimage import io, exposure, img_as_ubyte, img_as_float
from tqdm import trange
import matplotlib.pyplot as plt
import argparse
import os


def firstOrderDerivative(n, k=1):
    return np.eye(n) * (-1) + np.eye(n, k=k)


def toeplitizMatrix(n, row):
    vecDD = np.zeros(n)
    vecDD[0] = 4
    vecDD[1] = -1
    vecDD[row] = -1
    vecDD[-1] = -1
    vecDD[-row] = -1
    return vecDD


def vectorize(matrix):
    return matrix.T.ravel()


def reshape(vector, row, col):
    return vector.reshape((row, col), order='F')


class LIME:
    def __init__(self, iterations=10, alpha=2, rho=2, gamma=0.7, strategy=2, *args, **kwargs):
        self.iterations = iterations
        self.alpha = alpha
        self.rho = rho
        self.gamma = gamma
        self.strategy = strategy

    def load(self, img):
        if isinstance(img, np.ndarray):
            # Convert the image to float if it's not already
            if img.dtype != np.float32 and img.dtype != np.float64:
                img = img_as_float(img)
            self.L = img
        else:
            raise ValueError("Input must be a numpy.ndarray")

        self.row = self.L.shape[0]
        self.col = self.L.shape[1]

        # Handle grayscale images (2D arrays) separately
        if self.L.ndim == 2:
            self.T_hat = self.L
        else:
            self.T_hat = np.max(self.L, axis=2)

        self.dv = firstOrderDerivative(self.row)
        self.dh = firstOrderDerivative(self.col, -1)
        self.vecDD = toeplitizMatrix(self.row * self.col, self.row)
        self.W = self.weightingStrategy()

    def weightingStrategy(self):
        if self.strategy == 2:
            dTv = self.dv @ self.T_hat
            dTh = self.T_hat @ self.dh
            Wv = 1 / (np.abs(dTv) + 1)
            Wh = 1 / (np.abs(dTh) + 1)
            return np.vstack([Wv, Wh])
        else:
            return np.ones((self.row * 2, self.col))

    def __T_subproblem(self, G, Z, u):
        X = G - Z / u
        Xv = X[:self.row, :]
        Xh = X[self.row:, :]
        temp = self.dv @ Xv + Xh @ self.dh
        numerator = fft.fft(vectorize(2 * self.T_hat + u * temp))
        denominator = fft.fft(self.vecDD * u) + 2
        T = fft.ifft(numerator / denominator)
        T = np.real(reshape(T, self.row, self.col))
        return exposure.rescale_intensity(T, (0, 1), (0.001, 1))

    def __G_subproblem(self, T, Z, u, W):
        dT = self.__derivative(T)
        epsilon = self.alpha * W / u
        X = dT + Z / u
        return np.sign(X) * np.maximum(np.abs(X) - epsilon, 0)

    def __Z_subproblem(self, T, G, Z, u):
        dT = self.__derivative(T)
        return Z + u * (dT - G)

    def __u_subproblem(self, u):
        return u * self.rho

    def __derivative(self, matrix):
        v = self.dv @ matrix
        h = matrix @ self.dh
        return np.vstack([v, h])

    def illumMap(self):
        T = np.zeros((self.row, self.col))
        G = np.zeros((self.row * 2, self.col))
        Z = np.zeros((self.row * 2, self.col))
        u = 1

        for _ in range(0, self.iterations):
            T = self.__T_subproblem(G, Z, u)
            G = self.__G_subproblem(T, Z, u, self.W)
            Z = self.__Z_subproblem(T, G, Z, u)
            u = self.__u_subproblem(u)

        return T ** self.gamma

    def enhance(self):
        self.T = self.illumMap()

        if self.L.ndim == 2:  # Grayscale image
            self.R = self.L / self.T  # Direct division
        else:  # Color image
            self.R = self.L / np.repeat(self.T[:, :, np.newaxis], 3, axis=2)

        self.R = exposure.rescale_intensity(self.R, (0, 1))
        self.R = img_as_ubyte(self.R)
        return self.R

In [28]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger('paddleocr').setLevel(logging.CRITICAL)
logging.getLogger('lime').setLevel(logging.CRITICAL)

In [34]:
iterations_list = [10,20] # [50, 100, 200]
alpha_list = [0.1, 0.5, 1.0, 1.5]
rho_list = [0.5, 1.0, 1.5]
gamma_list = [0.5, 1.0, 1.5]
strategy_list = [1,2] # [1, 2]

# Define your image file names
image_files = [f'test{i}.jpg' for i in range(1, 7)]
# Create a DataFrame to store results
dfs = []
i = 0
with tqdm(total=864) as pbar:
    # Iterate over different LIME parameter combinations
    for iterations in iterations_list:
        for alpha in alpha_list:
            for rho in rho_list:
                for gamma in gamma_list:
                    for strategy in strategy_list:
                        # Initialize LIME with current parameters
                        lime = LIME(iterations=iterations, alpha=alpha, rho=rho, gamma=gamma, strategy=strategy)
                        df_list = []
                        for image_file in image_files:
                            robot=""
                            if image_file == "test.jpg":
                                robot = "1391"
                            elif image_file == "test2.jpg":
                                robot = "2722"
                            elif image_file == "test3.jpg":
                                robot = "56"
                            elif image_file == "test4.jpg":
                                robot = "5401"
                            elif image_file == "test5.jpg":
                                robot = "8513"
                            elif image_file == "test6.jpg":
                                robot = "75"
                            
                            
                            # Load and process the image
                            image = cv2.imread(f'/notebooks/images/Robot/{image_file}')
                            height, width, _ = image.shape

                            # Extract and enhance the bottom half
                            bottom_half = image[height // 2:, :]
                            upscaled_bottom_half = upscale_model.upsample(bottom_half)
                            gray = cv2.cvtColor(upscaled_bottom_half, cv2.COLOR_BGR2GRAY)

                            # Apply LIME enhancement
                            lime.load(gray)
                            enhanced_image = lime.enhance()

                            # Perform OCR
                            result = ocr.ocr(np.array(enhanced_image), cls=True)
                            
                            try:
                                
                                # Extract OCR result and confidence
                                ocr_result = result[0][0][1][0] if result else 'No result'
                                confidence = result[0][0][1][1] if result else 0
                            except:
                                ocr_result = ""
                                confidence = 0.0
                            # Append results to DataFrame
                            df_list.append({
                                'robot': robot,
                                'iterations': iterations,
                                'alpha': alpha,
                                'rho': rho,
                                'gamma': gamma,
                                'strategy': strategy,
                                'image': image_file,
                                'ocr_result': ocr_result,
                                'confidence': confidence
                            })
                            pbar.update(1)
                            
                        df = pd.DataFrame(df_list)
                        dfs.append(df)
                        
                        

# send_msg("LIME Testing FINISHED", 100)         
# Save the results to a CSV file
results_df = pd.concat(dfs, ignore_index=True)
# Save the results to a CSV file
results_df.to_csv('lime_ocr_results.csv')

100%|██████████| 864/864 [31:13<00:00,  2.17s/it]


In [35]:
results_df = pd.concat(dfs, ignore_index=True)
# Save the results to a CSV file
results_df.to_csv('lime_resutls.csv')

In [1]:
import pandas as pd

In [32]:
df = pd.read_csv('lime_results_2.csv', index_col='Unnamed: 0')
filtered_df = df[df['alpha'].isin([1.5])]
filtered_df = filtered_df[filtered_df['rho'].isin([1.5])]
filtered_df = filtered_df[filtered_df['iterations'].isin([1])]
filtered_df = filtered_df[filtered_df['gamma'].isin([0.5])]
filtered_df = filtered_df[filtered_df['strategy'].isin([1])]
# filtered_df = df[df['robot'].isin([56])]

# Sort the DataFrame by the 'confidence' column in descending order
df_sorted = filtered_df.sort_values(by='confidence', ascending=False)
df_sorted.head(50)

,robot,iterations,alpha,rho,gamma,strategy,image,ocr_result,confidence
396,NaN,1,1.5,1.5,0.5,1,test1.jpg,1381,0.956357
401,75.0,1,1.5,1.5,0.5,1,test6.jpg,75,0.928284
399,5401.0,1,1.5,1.5,0.5,1,test4.jpg,SADI,0.865929
397,2722.0,1,1.5,1.5,0.5,1,test2.jpg,2720,0.831428
398,56.0,1,1.5,1.5,0.5,1,test3.jpg,5R,0.671939
400,8513.0,1,1.5,1.5,0.5,1,test5.jpg,NaN,0.000000


In [33]:
!pip3 install pyxDamerauLevenshtein

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyxDamerauLevenshtein: filename=pyxDamerauLevenshtein-1.8.0-cp39-cp39-macosx_10_9_x86_64.whl size=31398 sha256=78a2c96deb04d3f83fd3b6ef0a1722db32b19ef6124926a54ba3906aa72ddd74
  Stored in directory: /Users/shravanp/Library/Caches/pip/wheels/04/0d/db/f3f582a26b1b2eb5633ba14a80f417e0d9ea93a6472674651d
Successfully built pyxDamerauLevenshtein


In [7]:
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance_seqs, normalized_damerau_levenshtein_distance

In [14]:
print(normalized_damerau_levenshtein_distance_seqs('SADI', ["56", "8513", "5401"]))
# normalized_damerau_levenshtein_distance("sadi", "56")
print(levenshtein_distance("SADI", "56"), levenshtein_distance("SADI", "8513"), levenshtein_distance("SADI", "5401"))

[1.0, 1.0, 1.0]
4 4 4


In [ ]:
import numpy as np
from scipy import fft
from skimage import io, exposure, img_as_ubyte, img_as_float
from tqdm import trange
import matplotlib.pyplot as plt
import argparse
import os


def firstOrderDerivative(n, k=1):
    return np.eye(n) * (-1) + np.eye(n, k=k)


def toeplitizMatrix(n, row):
    vecDD = np.zeros(n)
    vecDD[0] = 4
    vecDD[1] = -1
    vecDD[row] = -1
    vecDD[-1] = -1
    vecDD[-row] = -1
    return vecDD


def vectorize(matrix):
    return matrix.T.ravel()


def reshape(vector, row, col):
    return vector.reshape((row, col), order='F')


class LIME:
    def __init__(self, iterations=10, alpha=2, rho=2, gamma=0.7, strategy=2, *args, **kwargs):
        self.iterations = iterations
        self.alpha = alpha
        self.rho = rho
        self.gamma = gamma
        self.strategy = strategy

    def load(self, img):
        if isinstance(img, np.ndarray):
            # Convert the image to float if it's not already
            if img.dtype != np.float32 and img.dtype != np.float64:
                img = img_as_float(img)
            self.L = img
        else:
            raise ValueError("Input must be a numpy.ndarray")

        self.row = self.L.shape[0]
        self.col = self.L.shape[1]

        # Handle grayscale images (2D arrays) separately
        if self.L.ndim == 2:
            self.T_hat = self.L
        else:
            self.T_hat = np.max(self.L, axis=2)

        self.dv = firstOrderDerivative(self.row)
        self.dh = firstOrderDerivative(self.col, -1)
        self.vecDD = toeplitizMatrix(self.row * self.col, self.row)
        self.W = self.weightingStrategy()

    def weightingStrategy(self):
        if self.strategy == 2:
            dTv = self.dv @ self.T_hat
            dTh = self.T_hat @ self.dh
            Wv = 1 / (np.abs(dTv) + 1)
            Wh = 1 / (np.abs(dTh) + 1)
            return np.vstack([Wv, Wh])
        else:
            return np.ones((self.row * 2, self.col))

    def __T_subproblem(self, G, Z, u):
        X = G - Z / u
        Xv = X[:self.row, :]
        Xh = X[self.row:, :]
        temp = self.dv @ Xv + Xh @ self.dh
        numerator = fft.fft(vectorize(2 * self.T_hat + u * temp))
        denominator = fft.fft(self.vecDD * u) + 2
        T = fft.ifft(numerator / denominator)
        T = np.real(reshape(T, self.row, self.col))
        return exposure.rescale_intensity(T, (0, 1), (0.001, 1))

    def __G_subproblem(self, T, Z, u, W):
        dT = self.__derivative(T)
        epsilon = self.alpha * W / u
        X = dT + Z / u
        return np.sign(X) * np.maximum(np.abs(X) - epsilon, 0)

    def __Z_subproblem(self, T, G, Z, u):
        dT = self.__derivative(T)
        return Z + u * (dT - G)

    def __u_subproblem(self, u):
        return u * self.rho

    def __derivative(self, matrix):
        v = self.dv @ matrix
        h = matrix @ self.dh
        return np.vstack([v, h])

    def illumMap(self):
        T = np.zeros((self.row, self.col))
        G = np.zeros((self.row * 2, self.col))
        Z = np.zeros((self.row * 2, self.col))
        u = 1

        for _ in range(0, self.iterations):
            T = self.__T_subproblem(G, Z, u)
            G = self.__G_subproblem(T, Z, u, self.W)
            Z = self.__Z_subproblem(T, G, Z, u)
            u = self.__u_subproblem(u)

        return T ** self.gamma

    def enhance(self):
        self.T = self.illumMap()

        if self.L.ndim == 2:  # Grayscale image
            self.R = self.L / self.T  # Direct division
        else:  # Color image
            self.R = self.L / np.repeat(self.T[:, :, np.newaxis], 3, axis=2)

        self.R = exposure.rescale_intensity(self.R, (0, 1))
        self.R = img_as_ubyte(self.R)
        return self.R